<a href="https://colab.research.google.com/github/charurathour/Data-science-projects/blob/main/Market_Basket_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shanushah14/market-basket-dataset")

print("Path to dataset files:", path)

100%|██████████| 5.00k/5.00k [00:00<00:00, 5.48MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/shanushah14/market-basket-dataset/versions/1


In [7]:
import os
dataset_path='/root/.cache/kagglehub/datasets/shanushah14/market-basket-dataset/versions/1'
print(os.listdir(dataset_path))

['market_basket_dataset.csv']


In [8]:
import pandas as pd
df=pd.read_csv(dataset_path+'/market_basket_dataset.csv')
df.head()

,BillNo,Itemname,Quantity,Price,CustomerID
0,1000,Apples,5,8.30,52299
1,1000,Butter,4,6.06,11752
2,1000,Eggs,4,2.66,16415
3,1000,Potatoes,4,8.10,22889
4,1004,Oranges,2,7.26,52255


In [9]:
df.shape

(500, 5)

In [10]:
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BillNo      500 non-null    int64  
 1   Itemname    500 non-null    object 
 2   Quantity    500 non-null    int64  
 3   Price       500 non-null    float64
 4   CustomerID  500 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 19.7+ KB


In [12]:
df['Itemname']=df['Itemname'].str.strip()
df['BillNo']=df['BillNo'].astype('str')
df=df[~df['BillNo'].str.contains('C')]
df.head()

,BillNo,Itemname,Quantity,Price,CustomerID
0,1000,Apples,5,8.30,52299
1,1000,Butter,4,6.06,11752
2,1000,Eggs,4,2.66,16415
3,1000,Potatoes,4,8.10,22889
4,1004,Oranges,2,7.26,52255


In [13]:
mybasket=df.groupby(['BillNo','Itemname'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('BillNo')

In [16]:
mybasket.head()

Itemname,Apples,Bananas,Bread,Butter,Cereal,Cheese,Chicken,Coffee,Eggs,Juice,Milk,Onions,Oranges,Pasta,Potatoes,Sugar,Tea,Tomatoes,Yogurt
BillNo,,,,,,,,,,,,,,,,,,,
1000,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1005,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1008,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0
1011,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
def my_encode_units(x):
    if x<=0:
        return 0
    if x>=1:
        return 1
my_basket_sets=mybasket.applymap(my_encode_units)
my_basket_sets.head()

<ipython-input-17-48ff91343fe3>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  my_basket_sets=mybasket.applymap(my_encode_units)


Itemname,Apples,Bananas,Bread,Butter,Cereal,Cheese,Chicken,Coffee,Eggs,Juice,Milk,Onions,Oranges,Pasta,Potatoes,Sugar,Tea,Tomatoes,Yogurt
BillNo,,,,,,,,,,,,,,,,,,,
1000,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1005,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0
1008,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1011,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Training model

In [25]:
my_frequent_itemsets=apriori(my_basket_sets,min_support=0.05,use_colnames=True)

In [26]:
my_frequent_itemsets.tail()

,support,itemsets
54,0.078431,"(Oranges, Milk)"
55,0.065359,"(Potatoes, Oranges)"
56,0.052288,"(Oranges, Yogurt)"
57,0.058824,"(Potatoes, Tomatoes)"
58,0.058824,"(Sugar, Tomatoes)"


In [27]:
my_rules=association_rules(my_frequent_itemsets,metric='lift',min_threshold=1)

In [28]:
my_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Sugar),(Apples),0.169935,0.163399,0.058824,0.346154,2.118462,1.0,0.031056,1.279508,0.636045,0.214286,0.218450,0.353077
1,(Apples),(Sugar),0.163399,0.169935,0.058824,0.360000,2.118462,1.0,0.031056,1.296977,0.631076,0.214286,0.228976,0.353077
2,(Apples),(Yogurt),0.163399,0.169935,0.052288,0.320000,1.883077,1.0,0.024520,1.220684,0.560547,0.186047,0.180787,0.313846
3,(Yogurt),(Apples),0.169935,0.163399,0.052288,0.307692,1.883077,1.0,0.024520,1.208424,0.564961,0.186047,0.172476,0.313846
4,(Bananas),(Butter),0.241830,0.163399,0.058824,0.243243,1.488649,1.0,0.019309,1.105509,0.432950,0.169811,0.095439,0.301622


In [29]:
my_basket_sets['Bananas'].sum()

37

In [30]:
my_basket_sets['Butter'].sum()

25

In [36]:
my_rules[(my_rules['lift']>=2.5)&(my_rules['confidence']>=0.3)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
48,(Pasta),(Chicken),0.150327,0.130719,0.052288,0.347826,2.660870,1.0,0.032637,1.332898,0.734615,0.228571,0.249755,0.373913
49,(Chicken),(Pasta),0.130719,0.150327,0.052288,0.400000,2.660870,1.0,0.032637,1.416122,0.718045,0.228571,0.293846,0.373913
70,(Oranges),(Milk),0.189542,0.156863,0.078431,0.413793,2.637931,1.0,0.048699,1.438293,0.766129,0.292683,0.304731,0.456897
71,(Milk),(Oranges),0.156863,0.189542,0.078431,0.500000,2.637931,1.0,0.048699,1.620915,0.736434,0.292683,0.383065,0.456897
